In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

os.environ["LANGSMITH_TRACING"] = os.getenv("LANGSMITH_TRACING")
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("Gemini_Api_Key")
os.environ["serpapi_key"] = os.getenv("serp_api_key")


In [15]:
#sql db implementation
import sqlite3
from typing import List, Tuple

class NewsDatabase:
    def __init__(self, db_name: str = "news.db"):
        self.db_name = db_name
        self.create_table()
    
    def create_table(self):
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS News (
                    Slno INTEGER PRIMARY KEY AUTOINCREMENT,
                    Title TEXT NOT NULL,
                    Link TEXT NOT NULL,
                    Snippet TEXT,
                
                )
            ''')
            conn.commit()
    
    def insert_news(self, title: str, link: str, snippet:str) -> bool:
        try:
            with sqlite3.connect(self.db_name) as conn:
                cursor = conn.cursor()
                cursor.execute(
                    "INSERT INTO News (Title, Link,Snippet) VALUES (?, ?,?)",
                    (title, link,snippet)
                )
                conn.commit()
                return True
        except Exception as e:
            print(f"Error inserting news: {e}")
            return False
    
    def get_all_news(self) -> List[Tuple]:
        with sqlite3.connect(self.db_name) as conn:
            cursor = conn.cursor()
            cursor.execute("SELECT * FROM News")
            return cursor.fetchall()

serp api tool defnision

In [17]:
from typing import Optional
from langchain_core.tools import BaseTool
from serpapi import GoogleSearch

class SerpAPINewsTool(BaseTool):
    name:str = "serpapi_news_search"
    description: str = ("Useful for searching recent news articles about a specific topic. "
                       "Input should be a search query string. "
                       "First determine whether user is seeking for news or not, if yes then search for news articles related to the query. "
                       "For example if user asks for news about covid-19, then pass covid-19 to tool, even user doesn't mention news directly in query.")
    
    def _run(self, query: str) -> str:
        params = {
            "q": query,
            "api_key": os.environ["serpapi_key"],
            "tbm": "nws",
            "hl": "en",
            "gl": "In",
        }
        
        try:
            search = GoogleSearch(params)
            results = search.get_dict()
            
            if "news_results" in results:
                # Format the results
                formatted_results = []
                for article in results["news_results"][:10]:  # Limit to top 10 news results
                    title = article['title']
                    link = article.get('link', '')
                    snippet = article.get('snippet', '')
                    
                    # Store in database
                    self.db.insert_news(title, link,snippet)
                    
                    formatted_results.append(
                        f"Title: {title}\n"
                        f"Link: {link}\n"
                        f"Snippet: {snippet}\n"
                    )
                return formatted_results
            return "No news results found."
            
        except Exception as e:
            return f"Error performing news search: {str(e)}"

    def _arun(self, query: str) -> str:
        # Async implementation would go here
        raise NotImplementedError("Async version not implemented")

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults(max_results=2)
#serch_results = search.invoke("what is the weather in Chennai")

#print(serch_results)
#add serpapi tool
serpapi_news = SerpAPINewsTool()

tools = [search,serpapi_news]

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
#creating llm instance
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

f:\GitRepo\Langchain_Experiment\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Following 4 cells experiment with travily search

In [ ]:
from langchain_core.messages import HumanMessage
# response = llm.invoke([HumanMessage(content="what is the weather in Chennai")])
# print(response)

In [10]:
model_with_tools = llm.bind_tools(tools)

In [11]:
from langgraph.prebuilt import create_react_agent
agent_executor = create_react_agent(model_with_tools,tools)
# response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})
# response["messages"]


In [12]:
#test call
# response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})
# response["messages"]
# trying to get the weather using tool
# response = agent_executor.invoke({"messages": [HumanMessage(content="what is the weather in Chennai")]})
# response["messages"]

[HumanMessage(content='what is the weather in Chennai', additional_kwargs={}, response_metadata={}, id='02194fc5-1f57-45b1-ac72-a2e2cbd3d636'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in Chennai"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-e4c36b31-c0c3-4c57-90c1-89e2f62bf06f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Chennai'}, 'id': 'e34e4a70-b591-48bb-a697-4ef622c150dc', 'type': 'tool_call'}], usage_metadata={'input_tokens': 130, 'output_tokens': 12, 'total_tokens': 142, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Chennai\', \'region\': \'Tamil Nadu\', \'country\': \'India\', \'lat\': 13.0833, \'lon\': 80.2833, \'tz_id\': \'Asia/Kolkata\', \'localtime_epoc

Following cell experimnet with serpapi

In [13]:
response = agent_executor.invoke({
    "messages": [
        HumanMessage(content="What are the latest news about artificial intelligence?")
    ]
})
print(response["messages"])

[HumanMessage(content='What are the latest news about artificial intelligence?', additional_kwargs={}, response_metadata={}, id='3b408289-1b9d-4cea-80ab-f34bf3068750'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'serpapi_news_search', 'arguments': '{"query": "artificial intelligence"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-b5255484-95dd-44dd-9c6f-7c523d312f51-0', tool_calls=[{'name': 'serpapi_news_search', 'args': {'query': 'artificial intelligence'}, 'id': '0bd7a110-cd97-45a3-a115-1a958848319d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 133, 'output_tokens': 9, 'total_tokens': 142, 'input_token_details': {'cache_read': 0}}), ToolMessage(content="Title: Trump announces private-sector $500 billion investment in AI infrastructure\nLink: https://www.reuters.com/technology/artificial-intelligence/trump-announce-private-sector-ai-infrastructure-investmen